In [76]:
import piheaan as heaan
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import math
import numpy as np
import pandas as pd
import os

In [77]:
# set parameter
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context has paramter information
heaan.make_bootstrappable(context) # make parameter bootstrapable

# create and save keys
key_file_path = "./keys"
sk = heaan.SecretKey(context) # create secret key
os.makedirs(key_file_path, mode=0o775, exist_ok=True)
sk.save(key_file_path+"/secretkey.bin") # save secret key

key_generator = heaan.KeyGenerator(context, sk) # create public key
key_generator.gen_common_keys()
key_generator.save(key_file_path+"/") # save public key

# load secret key and public key
# When a key is created, it can be used again to save a new key without creating a new one
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load secret key
pk = heaan.KeyPack(context, key_file_path+"/") # load public key
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk) # to load piheaan basic function
dec = heaan.Decryptor(context) # for decrypt
enc = heaan.Encryptor(context) # for encrypt

In [78]:
def calculate_result(user_id,df,print_count):
    '''
        Function to calculate individual stock returns 
        and overall returns for an individual using homomorphic encryption.
    '''
    # Function for making the dictionary by User ID
    def making_dict(column_name):
        result = df.groupby('User ID')[column_name].apply(list).reset_index()
        user_dictionary = {}
        for index, row in result.iterrows():
            user_dictionary[row[0]] = row[1]
        return user_dictionary

    # price is Average Price
    user_dict_price = making_dict('price')
    user_dict_price_diff = making_dict('평균 가격 대비')
    user_dict_qty = making_dict('qty')

    # To decide log_slots and num_slots, find the max length of the user's data
    temp = []
    for i in user_dict_qty.keys():
        temp.append(len(user_dict_qty[i]))
    max_len = max(temp)
    max_len

    # Find the most near 2 powered number of max_len
    log_slots = 0
    num_slots = 2**log_slots
    while max_len > num_slots:
        log_slots += 1
        num_slots = 2**log_slots

    # When the data lenght is lower than num_slots, fill the values to 0 to make the length same.
    data_qty = user_dict_qty[user_id]
    actual_len = len(data_qty)
    data_qty_padded = [0] * num_slots
    for i in range(len(data_qty)):
        data_qty_padded[i] = data_qty[i]
        
    message_qty = heaan.Message(log_slots)
    for i in range(num_slots):
        message_qty[i] = data_qty_padded[i]

    data_price_diff = user_dict_price_diff[user_id]
    data_price_diff_padded = [0] * num_slots
    for i in range(len(data_price_diff)):
        data_price_diff_padded[i] = data_price_diff[i]
        
    message_price_diff = heaan.Message(log_slots)
    for i in range(num_slots):
        message_price_diff[i] = data_price_diff_padded[i]
        
    data_price = user_dict_price[user_id]
    data_price_padded = [0] * num_slots
    for i in range(len(data_price)):
        data_price_padded[i] = data_price[i]

    message_price = heaan.Message(log_slots)
    for i in range(num_slots):
        message_price[i] = data_price_padded[i]

    ciphertext_qty = heaan.Ciphertext(context) # qty 
    ciphertext_price_diff = heaan.Ciphertext(context) # price difference between current stock price and the user's stock's average price
    ciphertext_price = heaan.Ciphertext(context) # price (averge price of user's stock)

    # Encrypt the data
    enc.encrypt(message_qty, pk, ciphertext_qty)
    enc.encrypt(message_price_diff, pk, ciphertext_price_diff)
    enc.encrypt(message_price, pk, ciphertext_price)

    # (ciphertext * ciphertext)
    # qty * (price difference with stock's current price) -> The result value to be used in the numerator(분자)
    result_mult = heaan.Ciphertext(context)
    eval.mult(ciphertext_qty, ciphertext_price_diff, result_mult)

    result_mult_message = heaan.Message(log_slots)
    dec.decrypt(result_mult, sk, result_mult_message)
    if print_count == 0:
        print("qty * (price difference with stock's current price) is", result_mult_message)
        print()

    # (ciphertext * ciphertext)
    # qty * (price -> the price that user buy the stocks) -> The result value to be used in the denominator(분모)
    result_mult2 = heaan.Ciphertext(context)
    eval.mult(ciphertext_qty, ciphertext_price, result_mult2)

    result_mult_message2 = heaan.Message(log_slots)
    dec.decrypt(result_mult2, sk, result_mult_message2)
    if print_count == 0:
        print("qty * price(the average price that user buy stocks)", result_mult_message2)
        print()

    # Because the value of result_mult_message2 is too big, use the 0.01 to lower the values. We will make it to orginal values later.
    data = [0.01] * num_slots
    message_001 = heaan.Message(log_slots)
    for i in range(num_slots):
        message_001[i] = data[i]

    ciphertext_001 = heaan.Ciphertext(context) # 0.01만 있는거
    enc.encrypt(message_001, pk, ciphertext_001)

    # (ciphertext * ciphertext)
    # mult2 * 0.01 , Multiply all values by 0.01. This method is decided because the values are too large, causing incorrect results. The original values will be restored later.
    result_mult3 = heaan.Ciphertext(context)
    eval.mult(ciphertext_001, result_mult2, result_mult3)

    result_mult_message3 = heaan.Message(log_slots)
    dec.decrypt(result_mult3, sk, result_mult_message3)

    data = result_mult_message3
    message = heaan.Message(log_slots)
    for i in range(num_slots):
        message[i] = data[i]
        
    ciphertext = heaan.Ciphertext(context)
    result_inv = heaan.Ciphertext(context)
    
    enc.encrypt(message, pk, ciphertext)
    approx.inverse(eval, ciphertext, result_inv) 

    decryptor = heaan.Decryptor(context)
    result_inv_message = heaan.Message(log_slots)

    # Use inverse to calculate the profit of each stocks 
    decryptor.decrypt(result_inv, sk, result_inv_message)
    if print_count == 0:
        print("Inverse of the (qty * price) of each stock, 64 is the value for 0 so doesn't need to be considered", result_inv_message)
        print()

    # (message * message)
    # Calcualte the profit of each stock
    result_final_message = heaan.Message(log_slots)
    eval.mult(result_inv_message, result_mult_message, result_final_message)

    # (message * message)
    # To get it to the original value, multiply 0.01 for make it to original
    result_final_message2 = heaan.Message(log_slots)
    eval.mult(result_final_message, message_001, result_final_message2)

    final_result= [] # Users' each profit of the each stocks
    for i in range(actual_len):
        final_result.append(result_final_message2[i])

    total_result = sum(result_mult_message)/sum(result_mult_message2) # Total profit of users
    
    # Not to print always ^^ , We printed the value for TA's convenience
    if print_count == 0:
        print("Profit of each stock", final_result)
        print()
        print("Total profit of one user", total_result)
    
    return final_result, total_result

In [79]:
file_path = 'trader_list_1.csv'
df1 = pd.read_csv(file_path)

file_path = 'trader_list_2.csv'
df2 = pd.read_csv(file_path)

file_path = 'trader_list_3.csv'
df3 = pd.read_csv(file_path)

In [80]:
# Combine the DataFrame by rows
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

In [81]:
# Check the unique values of userid
unique_userids = combined_df['User ID'].unique()
company_combined = []

In [82]:
for name in unique_userids:
    print_count = 10000
    if name == "matthewmiller@naver.com":
        print_count = 0
    stock_profit, total_profit = calculate_result(name,combined_df,print_count)
    company_combined.append([name, total_profit])
company_combined.sort(key = lambda x:x[0])
company_combined

qty * (price difference with stock's current price) is [ (41850.000000+0.000000j), (-94550.000000+0.000000j), (40.000000+0.000000j), (-312620.000000+0.000000j), (9115200.000000+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]

qty * price(the average price that user buy stocks) [ (940950.000000+0.000000j), (1024550.000000+0.000000j), (3415.000000+0.000000j), (1761060.000000+0.000000j), (26965800.000000+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]

Inverse of the (qty * price) of each stock, 64 is the value for 0 so doesn't need to be considered [ (0.000106+0.000000j), (0.000098+0.000000j), (0.029283+0.000000j), (0.000057+0.000000j), (0.000004+0.000000j), ..., (64.000000+0.000000j), (64.000000+0.000000j), (64.000000+0.000000j), (64.000000+0.000000j), (64.000000+0.000000j) ]

Profit of each stock [(0.0444763271162127+0j), (-0.09

[['alexwilson3@yahoo.com', (-0.0027369952813597487+0j)],
 ['allisonnelson@gmail.com', (-0.015010722983036682+0j)],
 ['amandabrown@yahoo.com', (0.07262859854175498+0j)],
 ['andrewyoung@gmail.com', (0.011180409264520214+0j)],
 ['angelaroberts@naver.com', (-0.07089304686759733+0j)],
 ['ashleylopez@yahoo.com', (0.06599136695743515+0j)],
 ['ashleyrobinson@gmail.com', (-0.07638476715247416+0j)],
 ['bobbyking@gmail.com', (-0.04950639199568366+0j)],
 ['brandonallen@gmail.com', (0.011180409264520214+0j)],
 ['briancampbell@gmail.com', (0.055222893236190965+0j)],
 ['brianlee@naver.com', (-0.03872975602860862+0j)],
 ['caroladams@gmail.com', (-0.07638476715247416+0j)],
 ['chloebennett@gmail.com', (-0.01594060624808634+0j)],
 ['chrisr45@naver.com', (0.029306095043316653+0j)],
 ['danielkim@gmail.com', (0.1122956137900506+0j)],
 ['davidevans@naver.com', (0.029306095043316653+0j)],
 ['davidnguyen@yahoo.com', (0.06507671378031897+0j)],
 ['emilyjones@gmail.com', (-0.01594060624808634+0j)],
 ['emilyward@n

In [83]:
length = len(company_combined)
new_data = []
for values in company_combined:
    new_data.append(values[1])

In [84]:
# Find the most near 2 powered number of max_len
log_slots = 0
num_slots = 2**log_slots
while length > num_slots:
    log_slots += 1
    num_slots = 2**log_slots
    
new_data_padded = [0] * num_slots
for i in range(len(new_data)):
    new_data_padded[i] = new_data[i]
    
new_message = heaan.Message(log_slots)

for i in range(num_slots):
    new_message[i]=new_data_padded[i]

new_ciphertext = heaan.Ciphertext(context)
enc.encrypt(new_message, pk, new_ciphertext)

# Input range : -0.5 ~ 0.5
# Use the sort funtion to know the profit rank of the users.
ciphertext_out_sort = heaan.Ciphertext(context)
sort.sort(eval, new_ciphertext, ciphertext_out_sort, num_slots, False) # 4th boolean: 0: descending, 1 : ascending order

message_out_sort = heaan.Message(log_slots)

dec.decrypt(ciphertext_out_sort, sk, message_out_sort)

print("sort : ", message_out_sort)
profit_rank = []
for i in range(64):
    if i >= 33 and i <= 42: # the values that have 0 need to be deleted
        continue
    profit_rank.append(message_out_sort[i])

index pair in unitSort :0, 1
sort :  [ (0.147141+0.000000j), (0.112296+0.000000j), (0.085163+0.000000j), (0.085163+0.000000j), (0.072629+0.000000j), ..., (-0.070893+0.000000j), (-0.070893+0.000000j), (-0.076385+0.000000j), (-0.076385+0.000000j), (-0.076385+0.000000j) ]
index pair in unitSort :1, 2
index pair in unitSort :1, 1
index pair in unitSort :2, 4
index pair in unitSort :2, 2
index pair in unitSort :2, 1
index pair in unitSort :3, 8
index pair in unitSort :3, 4
index pair in unitSort :3, 2
index pair in unitSort :3, 1
index pair in unitSort :4, 16
index pair in unitSort :4, 8
index pair in unitSort :4, 4
index pair in unitSort :4, 2
index pair in unitSort :4, 1
index pair in unitSort :5, 32
index pair in unitSort :5, 16
index pair in unitSort :5, 8
index pair in unitSort :5, 4
index pair in unitSort :5, 2
index pair in unitSort :5, 1


# Profit Rank 
### 1 matthewmiller@naver.com
### 2 danielkim@gmail.com
### 3 emilyward@naver.com, lisajackson@naver.com

In [86]:
profit_rank
# For ease, we wrote the top three player!

[(0.14714149318562905+0j),
 (0.11229561379005144+0j),
 (0.08516291589956171+0j),
 (0.08516291589956171+0j),
 (0.07262859854175478+0j),
 (0.06599136695743447+0j),
 (0.06599136695743447+0j),
 (0.06599136695743447+0j),
 (0.06507671378031867+0j),
 (0.06507671378031867+0j),
 (0.06507671378031867+0j),
 (0.05522289323619128+0j),
 (0.05522289323619128+0j),
 (0.045852622559441426+0j),
 (0.045852622559441426+0j),
 (0.045852622559441426+0j),
 (0.045702875747765354+0j),
 (0.045702875747765354+0j),
 (0.043386839392379285+0j),
 (0.04010544875882155+0j),
 (0.04010544875882155+0j),
 (0.04010544875882155+0j),
 (0.03776972417363532+0j),
 (0.02930609504331689+0j),
 (0.02930609504331689+0j),
 (0.02930609504331689+0j),
 (0.013382897622586721+0j),
 (0.013382897622586721+0j),
 (0.011180409264520226+0j),
 (0.011180409264520226+0j),
 (0.006052908219404626+0j),
 (0.006052908219404626+0j),
 (0.0060529082194045545+0j),
 (-0.0027369952813597843+0j),
 (-0.015010722983036523+0j),
 (-0.015010722983036523+0j),
 (-0.01